# Library

In [10]:
import os
os.chdir('/home/pot/Desktop/web-scan')
from detector import DetectorTF2
import numpy as np
from helpers import ocr_helpers
from helpers import load_label_map as label_map_util
from helpers import visualization_utils as viz_utils

# Configuration

In [11]:
ROOT_PATH = '/home/pot/Desktop/'
PATH_TO_MODEL_DIR = ROOT_PATH + 'web-scan/models/discharge_record_content_tf2'
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
PATH_TO_LABELS = ROOT_PATH + 'web-scan/models/discharge_record_content_tf2/label_map.pbtxt'
PATH_TO_TRAIN_FOLDER = ROOT_PATH + 'web-scan/test_data/discharge_record_content'
PATH_TO_PRE_DATA_FOLDER = ROOT_PATH + 'web-scan/test_data/train_data/'
MIN_CONF_THRESH = 0.7
MIN_SCORE_THRESH = 0.7
NUM_CLASSES = 1

# Load Model

In [6]:
model = DetectorTF2(path_to_model=PATH_TO_MODEL_DIR, 
                    path_to_labels=PATH_TO_LABELS,
                    nms_threshold=MIN_CONF_THRESH, 
                    score_threshold=MIN_SCORE_THRESH)
model.load_model()

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Make data train for model detection area of field

In [9]:
import os
import glob
import pandas as pd
from shutil import copyfile
import cv2

images = glob.glob(PATH_TO_TRAIN_FOLDER + '/*jpg')+ glob.glob(PATH_TO_TRAIN_FOLDER + '/*JPG')
k = 1
for img in images:
    img = ocr_helpers.read_image_from_dir(img)
    img = np.asarray(img)
    image_with_detections = img.copy()
    detections = model.predict(img)
    detection_classes = detections['detection_classes']
    detection_boxes  = detections['detection_boxes']
    detection_scores  = detections['detection_scores']
    im_height, im_width = img.shape[:2]
    for i in range(len(detection_classes)):
        if detection_scores[i] < 0.7:
            continue
        ymin = int(detection_boxes[i][0]*im_height)
        xmin = int(detection_boxes[i][1]*im_width)
        ymax = int(detection_boxes[i][2]*im_height)
        xmax = int(detection_boxes[i][3]*im_width)
        break
    end = img[ymin:ymax, xmin:xmax]
    cv2.imwrite(PATH_TO_PRE_DATA_FOLDER + str(k) + '.jpg', end)
    k += 1

In [ ]:
# %matplotlib inline
# viz_utils.visualize_boxes_and_labels_on_image_array(
#       image_with_detections,
#       detections['detection_boxes'],
#       detections['detection_classes'],
#       detections['detection_scores'],
#       category_index,
#       use_normalized_coordinates=True,
#       max_boxes_to_draw=1,
#       min_score_thresh=0.1,
#       agnostic_mode=False)
# ocr_helpers.implt(image_with_detections)
# ocr_helpers.implt(img)